In [4]:
%%capture
!pip install openai langchain langchain_openai

In [5]:
import os
import pandas as pd
import pyarrow.parquet as pq
import re
import openai
import json
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
import yaml

In [6]:
parquet_file_path = '/home/ec2-user/SageMaker/genebanks/data/3_out_data/Ecogeographic survey of the target species-Costa Rica-2004.parquet'
df = pd.read_parquet(parquet_file_path, engine='pyarrow')
df.head()

,Nombre_Carpeta,Nombre_Imagen,Texto_Extraido,conteo_caracteres,Numero_Pagina
0,Ecogeographic survey of the target species-Cos...,page_1.png,Outreof st 1998\nNEW\n3151\n3156\n3152\n3126\n...,321,1
1,Ecogeographic survey of the target species-Cos...,page_2.png,Part 1 - Ecogeographic survey of the target sp...,4529,2
2,Ecogeographic survey of the target species-Cos...,page_3.png,2\nResults and discussion\nPhaseolus costarice...,4375,3
3,Ecogeographic survey of the target species-Cos...,page_4.png,"3\nof bush varieties, usually planted in Octob...",4127,4
4,Ecogeographic survey of the target species-Cos...,page_5.png,"4\nBrunner, B.R.& J.S. Beaver. 1989. Estimatio...",2682,5


In [4]:
parquet_file_path2 = '/home/ec2-user/SageMaker/genebanks/data/3_out_data/Germplasm collection-Colombia-1990.parquet'
df2 = pd.read_parquet(parquet_file_path2, engine='pyarrow')
#df2.head()

In [ ]:
config_file = '/home/ec2-user/SageMaker/genebanks/genebank-general/config/config.yml'
with open(config_file, 'r') as file:
    config = yaml.safe_load(file)

# Establecer las variables de entorno
for key, value in config.items():
    os.environ[key] = value

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openAIModelName = 'gpt-4-turbo'

In [6]:
llm_mod1 = ChatOpenAI(model_name = openAIModelName,
                        temperature=0.3)
llm_mod2 = ChatOpenAI(model_name = openAIModelName,
                        temperature=0.5)

In [7]:
# Concatenar el texto extraído de todas las páginas
text1 = '\n'.join(df['Texto_Extraido'].tolist())
text2 = '\n'.join(df2['Texto_Extraido'].tolist())
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs1 = [Document(page_content=x) for x in text_splitter.split_text(text1)]
docs2 = [Document(page_content=x) for x in text_splitter.split_text(text2)]

Created a chunk of size 4528, which is longer than the specified 1000
Created a chunk of size 4374, which is longer than the specified 1000
Created a chunk of size 4126, which is longer than the specified 1000
Created a chunk of size 2681, which is longer than the specified 1000
Created a chunk of size 3565, which is longer than the specified 1000
Created a chunk of size 1729, which is longer than the specified 1000
Created a chunk of size 3982, which is longer than the specified 1000
Created a chunk of size 2679, which is longer than the specified 1000
Created a chunk of size 2546, which is longer than the specified 1000
Created a chunk of size 4169, which is longer than the specified 1000
Created a chunk of size 4333, which is longer than the specified 1000
Created a chunk of size 4244, which is longer than the specified 1000
Created a chunk of size 3179, which is longer than the specified 1000
Created a chunk of size 2981, which is longer than the specified 1000
Created a chunk of s

In [31]:
len(text1)

22204

In [32]:
len(text2)

73388

In [8]:
template3 = """Actúa como observador de colectas que encuentra y extrae la información solicitada del texto. Te voy a proporcionar un texto largo extraído de un PDF que es un informe de colecta.
Necesito que extraigas TODA la información específica del texto y de todas las tablas donde encuentres altitud, latitud y longitud y la estructures en formato JSON que contenga todas las accesiones 
relacionadas en esa colección y asignar a cada una la información correspondiente de: colector, número de accesión, país, departamento, estatus, especie, nombre local, municipio, fuente, fecha, lugar, altitud, longitud, latitud. 
Puede haber más información o menos información que la mencionada. El objetivo que no debes dejar pasar es el identificador de la semilla, altitud, longitud y latitud en TODO el documento y tablas.

Este es un ejemplo de como deberia resultar la data de una accesión:

    "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
    "numero_accesión": 2928,
    "pais": "CLB",
    "departamento": "HUILA",
    "municipio": "SANTA MARIA",
    "especie": "P.lunatus",
    "lugar": "7 Km W Puerto El Socorro, Las Delicias",
    "altitud": 1030,
    "longitud": "75.34W",
    "latitud": "02.57N"

Aquí está el texto real del PDF que necesito que proceses:"""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template3),
    ("human", human_template),
])

In [9]:
messages1 = chat_prompt.format_messages(text=docs1)
result1 = llm_mod1.invoke(messages1)
output1 = result1.content
print(result1.content)

```json
[
    {
        "numero_accesión": 3165,
        "especie": "leptostachyus",
        "pais": "Costa Rica",
        "departamento": "Cartago",
        "municipio": "San Nicolas, Quircot",
        "altitud": 1540,
        "longitud": "83°56'W",
        "latitud": "9°54'N"
    },
    {
        "numero_accesión": 3166,
        "especie": "oligospermus",
        "pais": "Costa Rica",
        "departamento": "Cartago",
        "municipio": "Tobosi, Tablon",
        "altitud": 1470,
        "longitud": "84°02'W",
        "latitud": "9°49'N"
    },
    {
        "numero_accesión": 3167,
        "especie": "leptostachyus",
        "pais": "Costa Rica",
        "departamento": "Cartago",
        "municipio": "Tobosi, Tablon",
        "altitud": 1470,
        "longitud": "84°02'W",
        "latitud": "9°49'N"
    },
    {
        "numero_accesión": 3168,
        "especie": "vulgaris silv.",
        "pais": "Costa Rica",
        "departamento": "San Jose",
        "municipio": "Sta. Maria 

In [10]:
messages2 = chat_prompt.format_messages(text=docs2)
result2 = llm_mod1.invoke(messages2)
output2 = result2.content
print(result2.content)

```json
[
    {
        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
        "numero_accesión": 2928,
        "pais": "CLB",
        "departamento": "HUILA",
        "municipio": "SANTA MARIA",
        "especie": "P. lunatus",
        "lugar": "7 Km W Puerto El Socorro, Las Delicias",
        "altitud": 1030,
        "longitud": "75.34W",
        "latitud": "02.57N"
    },
    {
        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
        "numero_accesión": 2929,
        "pais": "CLB",
        "departamento": "HUILA",
        "municipio": "SANTA MARIA",
        "especie": "V. venusta",
        "lugar": "9 Km W Puerto El Socorro, La Esperanza",
        "altitud": 1030,
        "longitud": "75.34W",
        "latitud": "02.57N"
    },
    {
        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
        "numero_accesión": 2930,
        "pais": "CLB",
        "departamento": "HUILA",
        "municipio": "SANTA MARIA",
        "especie": "V. candida",
        "lugar

In [11]:
messages3 = chat_prompt.format_messages(text=text2)
result3 = llm_mod1.invoke(messages3)
output3 = result3.content
print(result3.content)

```json
[
    {
        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
        "numero_accesión": 2928,
        "pais": "CLB",
        "departamento": "HUILA",
        "municipio": "SANTA MARIA",
        "especie": "P.lunatus",
        "lugar": "7 Km W Puerto El Socorro, Las Delicias",
        "altitud": 1030,
        "longitud": "75.34W",
        "latitud": "02.57N"
    },
    {
        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
        "numero_accesión": 2901,
        "pais": "CLB",
        "departamento": "CAUCA",
        "municipio": "INZA",
        "especie": "P.vulgaris",
        "nombre_local": "comun",
        "lugar": "2 Km E Inza",
        "altitud": 1660,
        "longitud": "76.02W",
        "latitud": "02.33N"
    },
    {
        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
        "numero_accesión": 2902,
        "pais": "CLB",
        "departamento": "CAUCA",
        "municipio": "PAEZ",
        "especie": "P. lunatus",
        "nombre_local":

In [12]:
template4 = """Actúa como observador de colectas que extrae información solicitada del texto. Te voy a proporcionar un texto largo extraído de un informe de colecta.
Extrae TODA la información específica donde encuentres altitud, latitud y longitud y la estructures en formato JSON que contenga todas las accesiones 
relacionadas en ese documento y asignar a cada una la información correspondiente de: colector, número de accesión, país, departamento, estatus, especie, nombre local, municipio, fuente, fecha, lugar, altitud, longitud, latitud. 
Puede haber más información o menos información que la mencionada. El objetivo que no debes dejar pasar es el identificador, altitud, longitud y latitud en TODO el documento y tablas.

Este es un ejemplo de como deberia resultar la data de una accesión:

    "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
    "numero_accesión": 2928,
    "pais": "CLB",
    "departamento": "HUILA",
    "municipio": "SANTA MARIA",
    "especie": "P.lunatus",
    "lugar": "7 Km W Puerto El Socorro, Las Delicias",
    "altitud": 1030,
    "longitud": "75.34W",
    "latitud": "02.57N"

Extrae todo lo encontrado.

Aquí está el texto real del PDF que necesito que proceses:"""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template4),
    ("human", human_template),
])

In [13]:
messages4 = chat_prompt.format_messages(text=text2)
result4 = llm_mod1.invoke(messages4)
output4 = result4.content
print(result4.content)

```json
[
    {
        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
        "numero_accesión": 2901,
        "pais": "CLB",
        "departamento": "CAUCA",
        "especie": "P.vulgaris",
        "nombre_local": "comun",
        "municipio": "INZA",
        "fuente": "potager",
        "fecha": "23/08/1990",
        "lugar": "2 Km E Inza",
        "altitud": 1660,
        "longitud": "76.02W",
        "latitud": "02.33N"
    },
    {
        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
        "numero_accesión": 2902,
        "pais": "CLB",
        "departamento": "CAUCA",
        "especie": "P. lunatus",
        "nombre_local": "cacha",
        "municipio": "PAEZ",
        "fuente": "potager",
        "fecha": "23/08/1990",
        "lugar": "1 Km S Guadualejo",
        "altitud": 1230,
        "longitud": "75.59W",
        "latitud": "02.32N"
    },
    {
        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
        "numero_accesión": 2903,
        "pais":

In [14]:
output4

'```json\n[\n    {\n        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",\n        "numero_accesión": 2901,\n        "pais": "CLB",\n        "departamento": "CAUCA",\n        "especie": "P.vulgaris",\n        "nombre_local": "comun",\n        "municipio": "INZA",\n        "fuente": "potager",\n        "fecha": "23/08/1990",\n        "lugar": "2 Km E Inza",\n        "altitud": 1660,\n        "longitud": "76.02W",\n        "latitud": "02.33N"\n    },\n    {\n        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",\n        "numero_accesión": 2902,\n        "pais": "CLB",\n        "departamento": "CAUCA",\n        "especie": "P. lunatus",\n        "nombre_local": "cacha",\n        "municipio": "PAEZ",\n        "fuente": "potager",\n        "fecha": "23/08/1990",\n        "lugar": "1 Km S Guadualejo",\n        "altitud": 1230,\n        "longitud": "75.59W",\n        "latitud": "02.32N"\n    },\n    {\n        "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",\n        "nume

In [15]:
len(output4)

11728

In [44]:
openAIModelName2 = 'gpt-4o'
llm_mod3 = ChatOpenAI(model_name = openAIModelName2,
                        temperature=0.3, streaming = False, model_kwargs={'max_tokens': 90000})

ValidationError: 1 validation error for ChatOpenAI
__root__
  Parameters {'max_tokens'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter. (type=value_error)

In [43]:
messages5 = chat_prompt.format_messages(text=docs2)
result5 = llm_mod3.invoke(messages5)
output5 = result5.content
print(result5.content)

TypeError: Completions.create() got an unexpected keyword argument 'context_length'

In [35]:
print(result5.content)

```json
[
  {
    "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
    "numero_accesión": 2901,
    "pais": "CLB",
    "departamento": "CAUCA",
    "estatus": "cult",
    "especie": "P.vulgaris",
    "nombre_local": "comun",
    "municipio": "INZA",
    "fuente": "potager",
    "fecha": "23/08/1990",
    "lugar": "2 Km E Inza",
    "altitud": 1660,
    "longitud": "76.02W",
    "latitud": "02.33N"
  },
  {
    "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
    "numero_accesión": 2902,
    "pais": "CLB",
    "departamento": "CAUCA",
    "estatus": "cult",
    "especie": "P. lunatus",
    "nombre_local": "cacha",
    "municipio": "PAEZ",
    "fuente": "potager",
    "fecha": "23/08/1990",
    "lugar": "1 Km S Guadualejo",
    "altitud": 1230,
    "longitud": "75.59W",
    "latitud": "02.32N"
  },
  {
    "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",
    "numero_accesión": 2903,
    "pais": "CLB",
    "departamento": "CAUCA",
    "estatus": "cult",
    "especie": "P. l

In [34]:
result5.response_metadata

{'finish_reason': 'length'}

In [33]:
result5

AIMessage(content='```json\n[\n  {\n    "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",\n    "numero_accesión": 2901,\n    "pais": "CLB",\n    "departamento": "CAUCA",\n    "estatus": "cult",\n    "especie": "P.vulgaris",\n    "nombre_local": "comun",\n    "municipio": "INZA",\n    "fuente": "potager",\n    "fecha": "23/08/1990",\n    "lugar": "2 Km E Inza",\n    "altitud": 1660,\n    "longitud": "76.02W",\n    "latitud": "02.33N"\n  },\n  {\n    "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",\n    "numero_accesión": 2902,\n    "pais": "CLB",\n    "departamento": "CAUCA",\n    "estatus": "cult",\n    "especie": "P. lunatus",\n    "nombre_local": "cacha",\n    "municipio": "PAEZ",\n    "fuente": "potager",\n    "fecha": "23/08/1990",\n    "lugar": "1 Km S Guadualejo",\n    "altitud": 1230,\n    "longitud": "75.59W",\n    "latitud": "02.32N"\n  },\n  {\n    "colectores": "DEBOUCK DG LOPEZ L ANGULO N & TORO O",\n    "numero_accesión": 2903,\n    "pais": "CLB",\n    "departamen

In [18]:
text_file = open("Output.txt", "w")
text_file.write(output5)
text_file.close()

In [21]:
help(ChatOpenAI)

Help on class ChatOpenAI in module langchain_openai.chat_models.base:

class ChatOpenAI(BaseChatOpenAI)
 |  ChatOpenAI(*, name: Optional[str] = None, cache: ForwardRef('Union[BaseCache, bool, None]') = None, verbose: bool = None, callbacks: ForwardRef('Callbacks') = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, custom_get_token_ids: Optional[Callable[[str], List[int]]] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, client: Any = None, async_client: Any = None, model: str = 'gpt-3.5-turbo', temperature: float = 0.7, model_kwargs: Dict[str, Any] = None, api_key: Optional[pydantic.v1.types.SecretStr] = None, base_url: Optional[str] = None, organization: Optional[str] = None, openai_proxy: Optional[str] = None, timeout: Union[float, Tuple[float, float], Any, NoneType] = None, max_retries: int = 2, streaming: bool = False, n: int = 1, max_tokens: Optional[int] = None, tiktoken_model_name: Optional[str] = Non

In [29]:
import langchain

In [30]:
langchain.__version__

'0.2.0'